# Optimizing Mfanalysis

## Toy data

Let's first get some toy data to test out timings

In [1]:
import mne

In [2]:
raw = mne.io.read_raw_fif('/media/merlin/Media/ScaledTime/MEG/eb180237/eb180237_ScaledTime_RS01_filtered_ica_sss_raw.fif')

Opening raw data file /media/merlin/Media/ScaledTime/MEG/eb180237/eb180237_ScaledTime_RS01_filtered_ica_sss_raw.fif...
    Range : 490500 ... 1234499 =    245.250 ...   617.250 secs
Ready.
Current compensation grade : 0


In [3]:
data = raw[:]

In [4]:
del raw

In [5]:
data = data[0]

## Wavelet analysis

In [1]:
from importlib import reload

In [2]:
import mfanalysis.wavelet

In [3]:
reload(mfanalysis.wavelet)

<module 'mfanalysis.wavelet' from '/home/merlin/Multifracs/mfanalysis/mfanalysis/wavelet.py'>

In [4]:
from mfanalysis.wavelet import wavelet_analysis

In [5]:
data.shape

NameError: name 'data' is not defined

In [11]:
p_exp = 2
wt_name = 'db3'
j1 = 1
j2 = 10
gamint = 1.0

In [14]:
from megfractal.utils import scale2freq

ModuleNotFoundError: No module named 'megfractal'

In [55]:
%%prun -T "profile.txt"

wavelet_analysis(data[0], p_exp, wt_name, j1, j2, gamint)

 
*** Profile printout saved to text file '"profile.txt"'. 


         2416 function calls (2354 primitive calls) in 0.039 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    0.007    0.000    0.015    0.001 wavelet.py:109(filtering)
       32    0.006    0.000    0.006    0.000 {built-in method numpy.core._multiarray_umath.correlate}
        1    0.006    0.006    0.039    0.039 wavelet.py:198(wavelet_analysis)
       85    0.004    0.000    0.004    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  179/117    0.004    0.000    0.013    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       48    0.003    0.000    0.003    0.000 utils.py:7(smart_power)
       16    0.002    0.000    0.012    0.001 wavelet.py:167(_compute_leaders)
       34    0.002    0.000    0.002    0.000 {built-in method numpy.arange}
        1    0.002    0.002    0.003    0.003 structurefunction.py:46(_compute)
        1    0.001    0.001    0.004    0.004 wavelet.py:37(_

In [12]:
%%timeit

wavelet_analysis(data[0], p_exp, wt_name, j1, j2, gamint)

42.3 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit

wavelet_analysis(data[0], p_exp, wt_name, j1, j2, gamint)

31.4 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
import numpy as np

In [17]:
np.show_config()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
    libraries = ['blis', 'blis']
    library_dirs = ['/usr/local/lib']
    define_macros = [('HAVE_CBLAS', None)]
    include_dirs = ['/usr/local/include']
    language = c
    runtime_library_dirs = ['/usr/local/lib']
blas_opt_info:
    libraries = ['blis', 'blis']
    library_dirs = ['/usr/local/lib']
    define_macros = [('HAVE_CBLAS', None)]
    include_dirs = ['/usr/local/include']
    language = c
    runtime_library_dirs = ['/usr/local/lib']
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
  NOT AVAILABLE
openblas_clapack_info:
  NOT AVAILABLE
flame_info:
    libraries = ['flame', 'blis', 'blis', 'flame']
    library_dirs = ['/usr/local/lib', '/usr/local/lib']
    define_macros = [('HAVE_CBLAS', None)]
    include_dirs = ['/usr/local/include']
    language = c
    runtime_library_dirs = ['/usr/local/lib']
lapack_opt_info:
    libraries = ['flame', 'blis', 'blis', 'flame']
    library_dirs = ['/usr/local/lib', '/usr/local/lib']
    

In [14]:
%%prun
wavelet_analysis(data[0], p_exp, wt_name, j1, j2, gamint)

         2416 function calls (2354 primitive calls) in 0.047 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       32    0.010    0.000    0.010    0.000 {built-in method numpy.core._multiarray_umath.correlate}
        1    0.007    0.007    0.047    0.047 wavelet.py:198(wavelet_analysis)
       16    0.006    0.000    0.018    0.001 wavelet.py:109(filtering)
       85    0.006    0.000    0.006    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  179/117    0.004    0.000    0.016    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       48    0.003    0.000    0.003    0.000 utils.py:7(smart_power)
       16    0.003    0.000    0.014    0.001 wavelet.py:167(_compute_leaders)
        1    0.003    0.003    0.005    0.005 structurefunction.py:46(_compute)
       34    0.002    0.000    0.002    0.000 {built-in method numpy.arange}
        1    0.001    0.001    0.006    0.006 wavelet.py:37(_

## Full multifractal

In [16]:
from mfanalysis.mf_analysis import mf_analysis_full

In [17]:
import numpy as np

In [18]:
# l = np.logspace(np.log10(0.01), np.log10(5), 10)
# q = np.array([*-l, *l, 0, 1, 2, -1, -2])
# q.sort()
q = np.array([-2, -1, 0, 1, 2])
n_cumul = 2

In [17]:
%%timeit

mf_analysis_full(data[0], j1, j2, gamint=1.0, wt_name=wt_name, p_exp=p_exp, q=q, n_cumul=n_cumul)

1.8 s ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit

mf_analysis_full(data[0], j1, j2, gamint=1.0, wt_name=wt_name, p_exp=p_exp, q=q, n_cumul=n_cumul)

1.83 s ± 51.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit

mf_analysis_full(data[0], j1, j2, gamint=1.0, wt_name=wt_name, p_exp=p_exp, q=q, n_cumul=n_cumul)

275 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Multifractal

In [34]:
import mfanalysis.mfspectrum as mfs

In [35]:
reload(mfs)

<module 'mfanalysis.mfspectrum' from '/home/merlin/Multifracs/mfanalysis/mfanalysis/mfspectrum.py'>

In [36]:
import mfanalysis.mf_analysis

In [37]:
reload(mfanalysis.mf_analysis)

<module 'mfanalysis.mf_analysis' from '/home/merlin/Multifracs/mfanalysis/mfanalysis/mf_analysis.py'>

In [39]:
from mfanalysis.mf_analysis import minimal_mf_analysis

In [38]:
from mfanalysis.mf_analysis import mf_analysis

In [22]:
wt_transform = wavelet_analysis(data[0], p_exp, wt_name, j1, j2, gamint=1.0)

In [23]:
%%timeit

mf_analysis(wt_transform.wt_coefs,
            wt_transform.wt_leaders,
            wt_transform.j2_eff,
            p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

1.82 s ± 73.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit

mf_analysis(wt_transform.wt_coefs,
            wt_transform.wt_leaders,
            wt_transform.j2_eff,
            p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

1.95 s ± 94.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit

mf_analysis(wt_transform.wt_coefs,
            wt_transform.wt_leaders,
            wt_transform.j2_eff,
            p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

1.93 s ± 68.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
%%timeit

mf_analysis(wt_transform.wt_coefs,
            wt_transform.wt_leaders,
            wt_transform.j2_eff,
            p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

1.87 s ± 68.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit

mf_analysis(wt_transform.wt_coefs,
            wt_transform.wt_leaders,
            wt_transform.j2_eff,
            p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

1.67 s ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit

mf_analysis(wt_transform.wt_coefs,
            wt_transform.wt_leaders,
            wt_transform.j2_eff,
            p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

207 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%%timeit

minimal_mf_analysis(wt_transform.wt_coefs,
                    wt_transform.wt_leaders,
                    wt_transform.j2_eff,
                    p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

29.9 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%%prun -T "profile_mf.txt"

mf_analysis(wt_transform.wt_coefs,
            wt_transform.wt_leaders,
            wt_transform.j2_eff,
            p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

 
*** Profile printout saved to text file '"profile_mf.txt"'. 


         8121 function calls (8075 primitive calls) in 0.247 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.123    0.061    0.177    0.088 mfspectrum.py:44(_compute)
      320    0.057    0.000    0.057    0.000 utils.py:7(smart_power)
      568    0.017    0.000    0.017    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        2    0.016    0.008    0.018    0.009 cumulants.py:54(_compute)
        2    0.012    0.006    0.046    0.023 structurefunction.py:46(_compute)
  532/490    0.012    0.000    0.019    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        2    0.003    0.002    0.005    0.003 estimation.py:44(estimate_hmin)
       34    0.001    0.000    0.003    0.000 utils.py:43(linear_regression)
      224    0.001    0.000    0.005    0.000 _methods.py:134(_mean)
      224    0.000    0.000    0.006    0.000 fromnumeric.py:3231(mean)
      224    0.000    0.000   

In [42]:
%%prun -T "profile_mf.txt"

minimal_mf_analysis(wt_transform.wt_coefs,
                    wt_transform.wt_leaders,
                    wt_transform.j2_eff,
                    p_exp=p_exp, j1=j1, weighted=True, n_cumul=n_cumul, q=q)

 
*** Profile printout saved to text file '"profile_mf.txt"'. 


         2171 function calls in 0.033 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       80    0.015    0.000    0.015    0.000 utils.py:7(smart_power)
        1    0.007    0.007    0.008    0.008 cumulants.py:54(_compute)
        1    0.006    0.006    0.024    0.024 structurefunction.py:46(_compute)
      154    0.002    0.000    0.002    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      112    0.000    0.000    0.003    0.000 _methods.py:134(_mean)
        7    0.000    0.000    0.001    0.000 utils.py:43(linear_regression)
      112    0.000    0.000    0.003    0.000 fromnumeric.py:3231(mean)
      112    0.000    0.000    0.000    0.000 _methods.py:50(_count_reduce_items)
        1    0.000    0.000    0.033    0.033 {built-in method builtins.exec}
      154    0.000    0.000    0.003    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      121    0.000    0.000    0.000    0.00

## Timing wavelet analysis

In [24]:
wt_transform = wavelet_analysis(data[0], p_exp, wt_name, j1, j2, gamint=0.0)

In [45]:
%timeit np.power(2, 10)

1.41 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [46]:
%timeit 2**10

5.49 ns ± 0.083 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [25]:
%timeit np.power(2, 10)

1.48 µs ± 30.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
%timeit 2**10

5.13 ns ± 0.0965 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


Timing array ops

In [7]:
detail = np.random.randn(37200000)

Squaring

In [69]:
%timeit np.power(detail, 2)

6.97 ms ± 59.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [70]:
%timeit detail * detail

34.3 µs ± 2.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [128]:
%timeit np.square(detail)

30.5 µs ± 3.32 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [55]:
%timeit np.power(detail, 2)

711 ms ± 4.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
%timeit detail * detail

63.2 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%timeit np.square(detail)

92.2 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Rooting

In [136]:
%timeit np.power(np.abs(detail), 1/2)

7.54 ms ± 482 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [137]:
%timeit np.sqrt(np.abs(detail))

739 µs ± 31.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [58]:
%timeit np.power(np.abs(detail), 1/2)

712 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%timeit np.sqrt(np.abs(detail))

215 ms ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Raising to arbitrary power

In [60]:
q = 3

In [61]:
%timeit detail ** q

733 ms ± 32.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
%timeit np.power(detail, q)

761 ms ± 41.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
%%timeit

array_out = np.ones(detail.shape)

for _ in range(q):
    array_out *= detail

array_out

172 ms ± 4.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
q = 10

In [65]:
%timeit detail ** q

842 ms ± 50.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%timeit np.power(detail, q)

777 ms ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%%timeit

array_out = np.ones(detail.shape)

for _ in range(q):
    array_out *= detail

array_out

489 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
detail ** q

array([1.14985385e+03, 1.59984739e-08, 2.55269793e-03, ...,
       1.02818497e+03, 1.07067481e-06, 2.78035149e+01])

In [50]:
np.power(detail, q)

array([1.14985385e+03, 1.59984739e-08, 2.55269793e-03, ...,
       1.02818497e+03, 1.07067481e-06, 2.78035149e+01])

In [51]:
array_out = np.ones(detail.shape)

for _ in range(q):
    array_out *= detail

array_out

array([1.14985385e+03, 1.59984739e-08, 2.55269793e-03, ...,
       1.02818497e+03, 1.07067481e-06, 2.78035149e+01])

Computing mean

In [88]:
detail = np.random.randn(37200000)

In [74]:
%timeit detail.mean()

1.66 ms ± 25.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
%timeit np.mean(detail)

1.68 ms ± 28.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Testing smart power

In [56]:
import mfanalysis.utils

In [57]:
reload(mfanalysis.utils)

<module 'mfanalysis.utils' from '/home/merlin/Multifracs/mfanalysis/mfanalysis/utils.py'>

In [58]:
from mfanalysis.utils import smart_power

In [59]:
exponent = 0

In [63]:
%timeit smart_power(detail, exponent)

48.6 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
%timeit detail ** exponent

51.5 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [66]:
exponent = 1

In [67]:
%timeit smart_power(detail, exponent)

119 ns ± 1.74 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [68]:
%timeit detail ** exponent

67.6 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [69]:
exponent = 2

In [70]:
%timeit smart_power(detail, exponent)

65.7 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%timeit detail ** exponent

95 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
exponent = -1

In [73]:
%timeit smart_power(detail, exponent)

94.6 ms ± 2.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [74]:
%timeit detail ** exponent

94.5 ms ± 753 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
exponent = -2

In [76]:
%timeit smart_power(detail, exponent)

716 ms ± 7.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%timeit detail ** exponent

718 ms ± 7.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Summing

In [91]:
%timeit detail.sum()

16.5 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
%timeit np.sum(detail)

16.2 ms ± 574 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [94]:
%timeit detail2.sum()

371 µs ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [95]:
%timeit np.sum(detail2)

383 µs ± 23.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Filtering

In [37]:
approx = np.random.randn(20)

In [30]:
from scipy.signal import convolve

In [32]:
import pywt

In [33]:
wavelet = pywt.Wavelet(wt_name)
high_filter = -1*np.array(wavelet.dec_hi)
low_filter = np.array(wavelet.dec_lo)

In [42]:
nj_temp = len(approx)

In [46]:
approx[3] = np.inf

In [47]:
high = convolve(approx, high_filter, mode='full', method='direct')
low = convolve(approx, low_filter, mode='full', method='direct')

In [62]:
high_v = convolve(approx, high_filter, mode='valid', method='direct')
low_v = convolve(approx, low_filter, mode='valid', method='direct')

In [56]:
%timeit high = convolve(approx, high_filter, mode='full', method='direct')

4.61 µs ± 74.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [55]:
%timeit high = convolve(approx, high_filter, mode='full')

14.1 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [48]:
high

array([ 0.09333526, -0.47756927,  0.69726451,         inf,        -inf,
               inf,         inf,        -inf,        -inf,  0.47655917,
       -0.1049862 , -0.25457883, -0.19688186,  0.05075347, -0.15664647,
        1.07860753, -0.8505543 ,  0.08311984, -0.2749667 ,  0.36563768,
        0.27992021, -0.44301442, -0.03158715,  0.09498438,  0.0272934 ])

In [63]:
high_v

array([        inf,         inf,        -inf,        -inf,  0.47655917,
       -0.1049862 , -0.25457883, -0.19688186,  0.05075347, -0.15664647,
        1.07860753, -0.8505543 ,  0.08311984, -0.2749667 ,  0.36563768])

In [49]:
low

array([ 0.00988322, -0.05056953,  0.02229138,         inf,        -inf,
              -inf,         inf,         inf,         inf,  0.79713398,
        0.5697332 , -0.36751585, -0.93210171, -0.60682878,  0.11681667,
        0.60564466,  0.30999824, -0.88810852, -1.40611006, -0.78137356,
       -0.26944138, -0.49423913, -1.04589783, -0.89701481, -0.25775376])

In [57]:
# index of first good value
fp = len(high_filter) - 2
# index of last good value
lp = nj_temp - 1

In [64]:
# replace border with Inf
high[0:fp] = np.inf
high[lp+1:] = np.inf
low[0:fp] = np.inf
low[lp+1:] = np.inf

In [65]:
high

array([        inf,         inf,         inf,         inf,        -inf,
               inf,         inf,        -inf,        -inf,  0.47655917,
       -0.1049862 , -0.25457883, -0.19688186,  0.05075347, -0.15664647,
        1.07860753, -0.8505543 ,  0.08311984, -0.2749667 ,  0.36563768,
               inf,         inf,         inf,         inf,         inf])

In [ ]:
# centering and subsampling
detail_idx = np.arange(1, nj_temp + 1, 2)
approx_idx = np.arange(-1, nj_temp - 1, 2) + len(high_filter)

In [ ]:
detail = high[detail_idx]
approx = low[approx_idx]

In [35]:
detail

array([        inf,         inf, -1.67527062,  0.41893915,  0.06961611,
        1.58621069, -2.08857282,  0.43859429, -0.04792566, -0.68135377])

In [36]:
approx

array([ 0.78594536, -0.99804386, -1.10785982, -0.9057365 ,  1.9414605 ,
        0.52055171, -0.41227763,  1.5114668 ,         inf,         inf])

In [3]:
A = np.random.rand(1000, 100000)

In [4]:
B = np.random.rand(100000, 1000)

In [8]:
%%timeit
np.dot(A,B)

1.88 s ± 33.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
np.dot(A,B)

3.66 s ± 156 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [93]:
detail2 = np.random.randn(100, 10000)

In [70]:
%timeit detail2 * detail2

1.16 ms ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [71]:
%timeit detail2 ** 2

2.16 ms ± 67.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
%timeit np.power(detail2, 2)

18.7 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [76]:
3 * 2 ** 3

24

In [77]:
3 * (2 ** 3)

24

In [78]:
(3 * 2) ** 3

216